# Trust Region Example

A sample usage of the STEAM library for testing various trust-region solvers on a divergent (for Gauss-Newton)
error metric.

In [1]:
import typing
import numpy as np
np.set_printoptions(6, suppress=True)

from pysteam import evaluator, problem, state, solver

Define a simple error metric to test trust region methods. Implements a vector error in $\mathbb{R}^2$ with

$$
\mathbf{e} = \begin{bmatrix} x+1 \\ -2x^2 + x - 1 \end{bmatrix}
$$

Minimum error is at zero. Notably vanilla Gauss Newton is unable to converge to the answer as a step near zero causes it to diverge.


In [2]:
class DivergenceErrorEval(evaluator.Evaluator):

  def __init__(self, state_vec: state.VectorSpaceStateVar) -> None:
    super().__init__()
    assert state_vec.get_perturb_dim() == 1
    self._state_vec = state_vec

  def is_active(self) -> bool:
    return not self._state_vec.is_locked()

  def evaluate(self, lhs: typing.Optional[np.ndarray] = None):
    x = self._state_vec.get_value()[0, 0]
    eval = np.empty((2, 1))
    eval[0, 0] = x + 1.0
    eval[1, 0] = -2.0 * x * x + x - 1.0

    if lhs is None:
      return eval

    jacs = dict()

    if not self._state_vec.is_locked():
      jac = np.empty((2, 1))
      jac[0, 0] = 1.0
      jac[1, 0] = -4.0 * x + 1.0
      jacs = {self._state_vec.get_key(): lhs @ jac}

    return eval, jacs


def setup_divergence_problem():
  # Create vector state variable
  initial = np.array([[10.0]])
  state_var = state.VectorSpaceStateVar(initial)

  # Setup noise model and loss function
  noise_model = problem.StaticNoiseModel(np.eye(2))
  loss_func = problem.L2LossFunc()
  error_func = DivergenceErrorEval(state_var)
  cost_term = problem.WeightedLeastSquareCostTerm(error_func, noise_model, loss_func)

  # Initialize problem
  opt_prob = problem.OptimizationProblem()
  opt_prob.add_state_var(state_var)
  opt_prob.add_cost_term(cost_term)

  return opt_prob

In [3]:
gauss_newton = solver.GaussNewtonSolver(setup_divergence_problem(), verbose=True, max_iterations=100)
gauss_newton.optimize()
print("Gauss Newton terminates from:", gauss_newton.termination_cause, "after", gauss_newton.curr_iteration,
      "iterations.")


Begin Optimization
------------------
Number of States:  1
Number of Cost Terms:  1
Initial Cost:  18301.0
Iteration:    1  -  Cost:  1165.4162
Iteration:    2  -  Cost:    79.8090
Iteration:    3  -  Cost:     7.6464
Iteration:    4  -  Cost:     1.6152
Iteration:    5  -  Cost:     4.2988
Iteration:    6  -  Cost:     1.0289
Iteration:    7  -  Cost:     1.0416
Iteration:    8  -  Cost:     1.6583
Iteration:    9  -  Cost:     1.0277
Iteration:   10  -  Cost:     1.3380
Iteration:   11  -  Cost:     1.0526
Iteration:   12  -  Cost:     1.9861
Iteration:   13  -  Cost:     1.0116
Iteration:   14  -  Cost:     1.0948
Iteration:   15  -  Cost:     1.0634
Iteration:   16  -  Cost:     2.3784
Iteration:   17  -  Cost:     1.0022
Iteration:   18  -  Cost:     1.0119
Iteration:   19  -  Cost:     1.0244
Iteration:   20  -  Cost:     1.2767
Iteration:   21  -  Cost:     1.0578
Iteration:   22  -  Cost:     2.1688
Iteration:   23  -  Cost:     1.0061
Iteration:   24  -  Cost:     1.0409
Itera

In [4]:
ls_gauss_newton = solver.LineSearchGaussNewtonSolver(setup_divergence_problem(), verbose=True, max_iterations=100)
ls_gauss_newton.optimize()
print("Line Search GN terminates from:", ls_gauss_newton.termination_cause, "after", ls_gauss_newton.curr_iteration, "iterations.")

Begin Optimization
------------------
Number of States:  1
Number of Cost Terms:  1
Initial Cost:  18301.0
Iteration:    1  -  Cost:  1165.4162  -  Search Coeff:  1.000
Iteration:    2  -  Cost:    79.8090  -  Search Coeff:  1.000
Iteration:    3  -  Cost:     7.6464  -  Search Coeff:  1.000
Iteration:    4  -  Cost:     1.6152  -  Search Coeff:  1.000
Iteration:    5  -  Cost:     1.0583  -  Search Coeff:  0.500
Iteration:    6  -  Cost:     1.0561  -  Search Coeff:  1.000
Iteration:    7  -  Cost:     1.0009  -  Search Coeff:  0.250
Iteration:    8  -  Cost:     1.0002  -  Search Coeff:  0.500
Iteration:    9  -  Cost:     1.0000  -  Search Coeff:  0.500
Iteration:   10  -  Cost:     1.0000  -  Search Coeff:  0.500
Termination Cause:  CONVERGED ABSOLUTE CHANGE
Total Optimization Time: 0.0100 seconds
Line Search GN terminates from: CONVERGED ABSOLUTE CHANGE after 10 iterations.


In [5]:
lev_marq = solver.LevMarqGaussNewtonSolver(setup_divergence_problem(), verbose=True, max_iterations=100)
lev_marq.optimize()
print("Levenberg–Marquardt terminates from:", lev_marq.termination_cause, "after", lev_marq.curr_iteration,
      "iterations.")


Begin Optimization
------------------
Number of States:  1
Number of Cost Terms:  1
Initial Cost:  18301.0
Iteration:    1  -  Cost:  1165.4167  -  TR Shrink:  0.000  -  AvP Ratio:  0.937
Iteration:    2  -  Cost:    79.8091  -  TR Shrink:  0.000  -  AvP Ratio:  0.937
Iteration:    3  -  Cost:     7.6464  -  TR Shrink:  0.000  -  AvP Ratio:  0.936
Iteration:    4  -  Cost:     1.6152  -  TR Shrink:  0.000  -  AvP Ratio:  0.933
Iteration:    5  -  Cost:     1.0583  -  TR Shrink:  7.000  -  AvP Ratio:  0.477
Iteration:    6  -  Cost:     1.0001  -  TR Shrink:  1.000  -  AvP Ratio:  0.611
Iteration:    7  -  Cost:     1.0000  -  TR Shrink:  1.000  -  AvP Ratio:  0.318
Termination Cause:  CONVERGED ABSOLUTE CHANGE
Total Optimization Time: 0.0129 seconds
Levenberg–Marquardt terminates from: CONVERGED ABSOLUTE CHANGE after 7 iterations.


In [6]:
dl_gauss_newton = solver.DoglegGaussNewtonSolver(setup_divergence_problem(), verbose=True, max_iterations=100)
dl_gauss_newton.optimize()
print("Powell's Dogleg terminates from:", dl_gauss_newton.termination_cause, "after", dl_gauss_newton.curr_iteration,
      "iterations.")


Begin Optimization
------------------
Number of States:  1
Number of Cost Terms:  1
Initial Cost:  18301.0
Iteration:    1  -  Cost:  1165.4162  -  TR Shrink:  0.000  -  AvP Ratio:  0.937  -  Dogleg Segment: Gauss Newton   
Iteration:    2  -  Cost:    79.8090  -  TR Shrink:  0.000  -  AvP Ratio:  0.937  -  Dogleg Segment: Gauss Newton   
Iteration:    3  -  Cost:     7.6464  -  TR Shrink:  0.000  -  AvP Ratio:  0.936  -  Dogleg Segment: Gauss Newton   
Iteration:    4  -  Cost:     1.6152  -  TR Shrink:  0.000  -  AvP Ratio:  0.933  -  Dogleg Segment: Gauss Newton   
Iteration:    5  -  Cost:     1.0039  -  TR Shrink:  5.000  -  AvP Ratio:  0.657  -  Dogleg Segment: Grad Descent   
Iteration:    6  -  Cost:     1.0013  -  TR Shrink:  3.000  -  AvP Ratio:  0.278  -  Dogleg Segment: Grad Descent   
Iteration:    7  -  Cost:     1.0002  -  TR Shrink:  1.000  -  AvP Ratio:  0.411  -  Dogleg Segment: Grad Descent   
Iteration:    8  -  Cost:     1.0000  -  TR Shrink:  2.000  -  AvP Ratio: 